In [16]:
import geopandas as gpd
import ipyleaflet
from ipywidgets import HTML, Layout, FloatSlider, VBox, interact, widgets, Dropdown, Output, HBox, Button, SelectMultiple
from IPython.display import display
from ipyleaflet import Marker, MarkerCluster, Popup, Map, Polygon, Polyline, WidgetControl
import pandas as pd
from openrouteservice import Client
import openrouteservice.convert as convert
import re

In [26]:
# CARGAMOS LOS DATASETS QUE HEMOS CREADO

# Cargar los límites de los barrios (archivo GeoJSON)
barrios_geojson = "Barrios_valencia.geojson"
barrios_gdf = gpd.read_file(barrios_geojson)
# Ahora los pisos
pisos = pd.read_json('DATASET_FINAL.json')
# Por ultimo los POIs
colegios_con_barrio = pd.read_csv('colegios_por_barrio.csv')
hospitales_con_barrio = pd.read_csv('hospitales_por_barrio.csv')
parques_con_barrio = pd.read_csv('parques_por_barrio.csv')
buses_con_barrio = pd.read_csv('bus_por_barrio.csv')
tram_con_barrio = pd.read_csv('tram_por_barrio.csv')

In [69]:
# Barrios de los pisos de los cuales tenemos las rutas guardadas
barrios_disponibles = ['Albors', 'Beniferri', 'Camí Real', 'Ciutat Fallera', 'Ciutat Jardí', 'el Cabanyal - el Canyamelar', 
                      'Exposició', 'Faitanar', 'Jaume Roig', 'la Carrasca', 'la Creu Coberta', 'la Seu', 'Patraix', 'Poble Nou',
                      'Safranar', 'Sant Francesc', 'Trinitat', 'Vara de Quart', 'la Petxina']

# Definir el centro de Valencia
valencia_center = (39.4699, -0.3763)  # Coordenadas de Valencia

In [18]:
# Inicializar el cliente de OpenRouteService
ors_client = Client(key='5b3ce3597851110001cf624859af420fdd51467a9d1b5f39737c6113')

# Función para obtener la ruta más rápida en coche (profile='driving-car') o andando (profile='foot-walking')
def get_route(coords, profile='driving-car'):
    try:
        response = ors_client.directions(
            coordinates=coords,
            profile = profile,
            format='geojson'
        )
        if 'features' in response:
            # Acceso a las coordenadas directamente
            geometry = response['features'][0]['geometry']
            coordinates = geometry['coordinates']
            duration = response['features'][0]['properties']['segments'][0]['duration']


            # Verifica que las coordenadas sean una lista de puntos válidos
            if isinstance(coordinates, list) and all(isinstance(coord, list) and len(coord) == 2 for coord in coordinates):
                route_coords = [(point[1], point[0]) for point in coordinates]  # Intercambio de latitud y longitud
                return route_coords, duration
            else:
                print("Invalid coordinates format.")
                return [], 0
        else:
            print("No features found in the response.")
            return [], 0
    except Exception as e:
        print("Error fetching route:", e)
        return [], 0

In [19]:
# Función para eliminar rutas anteriores
def clear_current_routes():
    global current_routes
    for route in current_routes:
        m.remove_layer(route)
    current_routes = []

In [20]:
# Crear botones personalizados con estilos y texto dinámico
def create_button_with_style(label, icon, border_color, tooltip):
    return Button(
        description=label,
        icon=icon,
        layout=Layout(width="100%", padding="5px", border=f"2px solid {border_color}", margin="5px"),
        tooltip=tooltip,
        style={"button_color": "#f5f5f5", "font_weight": "bold"}
    )

In [21]:
# Actualizar los botones al seleccionar un piso
def update_buttons(row):
    if row['Barrio'] in barrios_disponibles:
        # Crear tiempos y agregar iconos dinámicamente
        school_time_car = f"🚗 {int(row['tiempo_colegio_coche'] // 60)} min {int(row['tiempo_colegio_coche'] % 60)} s"
        school_time_walk = f"🚶 {int(row['tiempo_colegio_andando'] // 60)} min {int(row['tiempo_colegio_andando'] % 60)} s"
    
        hospital_time_car = f"🚗 {int(row['tiempo_hospital_coche'] // 60)} min {int(row['tiempo_hospital_coche'] % 60)} s"
        hospital_time_walk = f"🚶 {int(row['tiempo_hospital_andando'] // 60)} min {int(row['tiempo_hospital_andando'] % 60)} s"
    
        park_time_car = f"🚗 {int(row['tiempo_parque_coche'] // 60)} min {int(row['tiempo_parque_coche'] % 60)} s"
        park_time_walk = f"🚶 {int(row['tiempo_parque_andando'] // 60)} min {int(row['tiempo_parque_andando'] % 60)} s"

        # Botón para colegios
        school_button.description = f"Colegio: {school_time_car} | {school_time_walk}"
        school_button.tooltip = f"Ruta al colegio más cercano: {school_time_car} (en coche), {school_time_walk} (andando)"
    
        # Botón para hospitales
        hospital_button.description = f"Hospital: {hospital_time_car} | {hospital_time_walk}"
        hospital_button.tooltip = f"Ruta al hospital más cercano: {hospital_time_car} (en coche), {hospital_time_walk} (andando)"
    
        # Botón para parques
        park_button.description = f"Parque: {park_time_car} | {park_time_walk}"
        park_button.tooltip = f"Ruta al parque más cercano: {park_time_car} (en coche), {park_time_walk} (andando)"
    else:
        # Botón para colegios
        school_button.description = f"Colegio"
        school_button.tooltip = f"Calcular ruta en coche y andando al colegio más cercano"
    
        # Botón para hospitales
        hospital_button.description = f"Hospital"
        hospital_button.tooltip = f"Calcular ruta en coche y andando al hospital más cercano"
    
        # Botón para parques
        park_button.description = f"Parque"
        park_button.tooltip = f"Calcular ruta en coche y andando al parque más cercano"

    # Conectar botones con las rutas del piso seleccionado
    connect_buttons_to_routes(row)

In [22]:
# Función para cerrar el panel
def close_panel_callback(change):
    output_panel.layout.display = "none"  # Ocultar el panel
    m.layout.width = "100%"               # Restaurar el mapa a pantalla completa

In [23]:
# Función para mostrar información en el panel al hacer clic en un marcador
def on_marker_click_output(event=None, type=None, coordinates=None, row=None):
    with details_output:
        details_output.clear_output(wait=True)  # Limpiar el contenido anterior
        display(HTML(f"""
        <h4>Detalles del piso seleccionado</h4>
        <b>Precio:</b> {row['PRICE']} €<br>
        <b>Habitaciones:</b> {row['ROOMNUMBER']}<br>
        <b>Baños:</b> {row['BATHNUMBER']}<br>
        <b>Metros cuadrados:</b> {row['CONSTRUCTEDAREA']} m²<br>
        <b>Año de contrucción:</b> {row['CONSTRUCTIONYEAR']}<br>
        <b>Hospital más cercano:</b> {row['hospital_mas_cercano']} ({row['distancia_al_hospital_mas_cercano']:.2f}m)<br>
        <b>Colegio más cercano:</b> {row['colegio_mas_cercano']} ({row['distancia_al_colegio_mas_cercano']:.2f}m)<br>
        <b>Parque más cercano:</b> {row['parque_mas_cercano']} ({row['distancia_al_parque_mas_cercano']:.2f}m)<br>
        <b>Distancia a la parada de bus más cercana:</b> {row['distancia_al_bus_mas_cercano']:.2f}m<br>
        <b>Distancia a la parada de tram más cercana:</b> {row['distancia_al_tram_mas_cercano']:.2f}m
        """))
    # Mostrar el panel flotante
    output_panel.layout.display = "flex"  # Mostrar el panel
    m.layout.width = "70%"               # Reducir el ancho del mapa para que quepa el panel
    # Actualizar los botones con las rutas del piso seleccionado
    update_buttons(row)

In [24]:
# Conectar botones a las rutas con el valor correcto de 'row'
def connect_buttons_to_routes(row):
    school_button.on_click(lambda change: show_routes("colegio", row))
    hospital_button.on_click(lambda change: show_routes("hospital", row))
    park_button.on_click(lambda change: show_routes("parque", row))

In [25]:
# Función para mostrar rutas
def show_routes(route_type, row):
    clear_current_routes()  # Eliminar las rutas anteriores

    # Mapeo de colores por tipo de ruta
    colors = {
        "colegio": "blue",
        "hospital": "red",
        "parque": "green",
    }

    # Tipo de lugar y color
    color = colors[route_type]

    if row['Barrio'] in barrios_disponibles:
        # Dibujar las rutas (en coche y andando)
        for mode in ["coche", "andando"]:
            route_key = f"ruta_{route_type}_{mode}"
            time_key = f"tiempo_{route_type}_{mode}"
    
            # Obtener datos de la ruta y el tiempo
            route = row[route_key]
            time = row[time_key]
    
            # Estilo de línea: continua para coche, discontinua para andando
            dash_array = None if mode == "coche" else "5, 10"
    
            # Dibujar la polilínea de la ruta
            polyline = ipyleaflet.Polyline(locations=route, color=color, fill=False, dash_array=dash_array)
            m.add_layer(polyline)
            current_routes.append(polyline)

    else:
        lat, lon = row['LATITUDE'], row['LONGITUDE']
        poi_lat = row[f'{route_type}_lat']
        poi_lon = row[f'{route_type}_lon']
        # Dibujar las rutas (en coche y andando)
        for mode in ["coche", "andando"]:
            # Obtener datos de la ruta y el tiempo llamando a la API
            if mode == "coche":
                route, time = get_route([(lon, lat), (poi_lon, poi_lat)], profile='driving-car')
            if mode == "andando":
                route, time = get_route([(lon, lat), (poi_lon, poi_lat)], profile='foot-walking')
    
            # Estilo de línea: continua para coche, discontinua para andando
            dash_array = None if mode == "coche" else "5, 10"
    
            # Dibujar la polilínea de la ruta
            polyline = ipyleaflet.Polyline(locations=route, color=color, fill=False, dash_array=dash_array)
            time_marker = ipyleaflet.Marker(
                location=route[len(route) // 2],  # Punto medio de la ruta
                draggable=False,
                icon=ipyleaflet.DivIcon(
                    html=f'<div style="color: {color}; font-style: bold; white-space: nowrap;">{mode.capitalize()}: {int(time//60)} min y {round(time%60, 2)} s</div>'
                )
            )
            m.add_layer(polyline)
            m.add_layer(time_marker)
            current_routes.append(polyline)
            current_routes.append(time_marker)

In [29]:
# Función para extraer latitud y longitud
def extract_coordinates(point):
    # Usar una expresión regular para extraer los números
    match = re.match(r'POINT \(([\d\.\-]+) ([\d\.\-]+)\)', point)
    if match:
        # Retorna los números como flotantes
        return float(match.group(1)), float(match.group(2))
    return None, None  # En caso de error

In [30]:
# Función para actualizar el mapa según el barrio seleccionado
def update_map(change):
    selected_barrios = barrio_selector.value
    selected_pisos = pisos  # DataFrame de pisos
    selected_colegios = colegios_con_barrio  # DataFrame de colegios
    selected_hospitales = hospitales_con_barrio  # DataFrame de hospitales
    selected_parques = parques_con_barrio  # DataFrame de parques
    selected_buses = buses_con_barrio  # DataFrame de paradas de bus
    selected_tram = tram_con_barrio  # DataFrame de paradas de tram

    # Limpiar los marcadores actuales del mapa
    for marker in markers:
        m.remove_layer(marker)
    markers.clear()

    # Filtrar los pisos según los barrios seleccionados
    if "Todos" in selected_barrios:
        filtered_pisos = selected_pisos
        filtered_colegios = selected_colegios
        filtered_hospitales = selected_hospitales
        filtered_parques = selected_parques
        filtered_buses = selected_buses
        filtered_tram = selected_tram
    else:
        filtered_pisos = selected_pisos[selected_pisos["Barrio"].isin(selected_barrios)]
        filtered_colegios = selected_colegios[selected_colegios["Barrio"].isin(selected_barrios)]
        filtered_hospitales = selected_hospitales[selected_hospitales["Barrio"].isin(selected_barrios)]
        filtered_parques = selected_parques[selected_parques["Barrio"].isin(selected_barrios)]
        filtered_buses = selected_buses[selected_buses["Barrio"].isin(selected_barrios)]
        filtered_tram = selected_tram[selected_tram["Barrio"].isin(selected_barrios)]

    # Añadir los marcadores de los pisos filtrados al mapa
    for _, row in filtered_pisos.iterrows():
        lat, lon = row["LATITUDE"], row["LONGITUDE"]
        marker = ipyleaflet.Marker(location=(lat, lon), draggable=False)
        marker.on_click(lambda event, type, coordinates, row=row: on_marker_click_output(event, type, coordinates, row))
        markers.append(marker)
        m.add_layer(marker)

    for _, row in filtered_colegios.iterrows():
        lon, lat = extract_coordinates(row['geometry'])
        name = row['name_left']
        popup_content = HTML()
        popup_content.value = f"<b>Colegio:</b> {name}"
        popup = ipyleaflet.Popup(location=(lat, lon), child=popup_content, close_button=False, auto_close=False, close_on_escape_key=False)
        marker = ipyleaflet.Marker(location=(lat, lon), draggable=False, icon=ipyleaflet.Icon(icon_url='http://maps.google.com/mapfiles/kml/shapes/schools_maps.png', icon_size=[30, 30]))
        marker.popup = popup
        markers.append(marker)
        m.add_layer(marker)

    for _, row in filtered_hospitales.iterrows():
        lon, lat = extract_coordinates(row['geometry'])
        name = row['name_left']
        popup_content = HTML()
        popup_content.value = f"<b>Hospital:</b> {name}"
        popup = ipyleaflet.Popup(location=(lat, lon), child=popup_content, close_button=False, auto_close=False, close_on_escape_key=False)
        marker = ipyleaflet.Marker(location=(lat, lon), draggable=False, icon=ipyleaflet.Icon(icon_url='http://maps.google.com/mapfiles/kml/shapes/hospitals_maps.png', icon_size=[30, 30]))
        marker.popup = popup
        markers.append(marker)
        m.add_layer(marker)

    for _, row in filtered_parques.iterrows():
        lon, lat = extract_coordinates(row['geometry'])
        name = row['name_left']
        popup_content = HTML()
        popup_content.value = f"<b>Parque:</b> {name}"
        popup = ipyleaflet.Popup(location=(lat, lon), child=popup_content, close_button=False, auto_close=False, close_on_escape_key=False)
        marker = ipyleaflet.Marker(location=(lat, lon), draggable=False, icon=ipyleaflet.Icon(icon_url='http://maps.google.com/mapfiles/kml/shapes/parks_maps.png', icon_size=[30, 30]))
        marker.popup = popup
        markers.append(marker)
        m.add_layer(marker)

    for _, row in filtered_buses.iterrows():
        lon, lat = extract_coordinates(row['geometry'])
        name = row['name_left']
        popup_content = HTML()
        popup_content.value = f"<b>Parada de Bus:</b> {name}"
        popup = ipyleaflet.Popup(location=(lat, lon), child=popup_content, close_button=False, auto_close=False, close_on_escape_key=False)
        marker = ipyleaflet.Marker(location=(lat, lon), draggable=False, icon=ipyleaflet.Icon(icon_url='http://maps.google.com/mapfiles/kml/shapes/bus.png', icon_size=[30, 30]))
        marker.popup = popup
        markers.append(marker)
        m.add_layer(marker)

    for _, row in filtered_tram.iterrows():
        lon, lat = extract_coordinates(row['geometry'])
        name = row['name_left']
        popup_content = HTML()
        popup_content.value = f"<b>Parada de Tram:</b> {name}"
        popup = ipyleaflet.Popup(location=(lat, lon), child=popup_content, close_button=False, auto_close=False, close_on_escape_key=False)
        marker = ipyleaflet.Marker(location=(lat, lon), draggable=False, icon=ipyleaflet.Icon(icon_url='http://maps.google.com/mapfiles/kml/shapes/tram.png', icon_size=[30, 30]))
        marker.popup = popup
        markers.append(marker)
        m.add_layer(marker)

In [ ]:
# Crear el mapa interactivo centrado en Valencia
m = ipyleaflet.Map(center=valencia_center, zoom=15, layout=Layout(width="100%", height="100vh"), scroll_wheel_zoom=True)

# Dibujar los límites de los barrios en el mapa
barrios_polygons = []
for _, barrio in barrios_gdf.iterrows():
    geometry = barrio["geometry"]
    
    # Si es un polígono o multipolígono, lo añadimos correctamente
    if geometry.geom_type == "Polygon":
        locations = [[(coord[1], coord[0]) for coord in geometry.exterior.coords]]
        polygon = Polygon(
            locations=locations,
            color="black",
            fill_color="gray",
            fill_opacity=0.2,
            name=barrio["name"]
        )
        m.add_layer(polygon)
        barrios_polygons.append(polygon)
    
    elif geometry.geom_type == "MultiPolygon":
        for poly in geometry.geoms:
            locations = [[(coord[1], coord[0]) for coord in poly.exterior.coords]]
            polygon = Polygon(
                locations=locations,
                color="black",
                fill_color="gray",
                fill_opacity=0.2,
                name=barrio["name"]
            )
            m.add_layer(polygon)
            barrios_polygons.append(polygon)
    
    # Si es un LineString, lo mostramos como una línea
    elif geometry.geom_type == "LineString":
        locations = [(coord[1], coord[0]) for coord in geometry.coords]
        polyline = Polyline(
            locations=locations,
            color="black",
            weight=2,
            fill=False
        )
        m.add_layer(polyline)

# Crear un selector múltiple para los barrios
barrio_selector = SelectMultiple(
    options=["Todos"] + list(barrios_gdf["name"].unique()),  # Lista de barrios
    value=["Todos"],
    description="Barrios:",
    layout=Layout(width="200px", height="150px"),  # Tamaño ajustado
    style={"description_width": "initial"}
)


# Crear el panel lateral (Output widget)
output_panel = VBox(
    layout=Layout(
        overflow='scroll hidden',
        display="none",       # Oculto inicialmente
        width="30%",          # Ancho del panel (30% del contenedor)
        height="100%",        # Altura completa
        background_color="white",
        border="1px solid #ccc",
        padding="10px",
    )
)

# Crear un botón para cerrar el panel
close_button = Button(
    description="Cerrar",
    layout=Layout(width="100%"),
    button_style="danger",  # Color rojo para indicar cierre
)

# Crear los botones iniciales
school_button = create_button_with_style("Colegio", "graduation-cap", "blue", "Mostrar la ruta al colegio más cercano")
hospital_button = create_button_with_style("Hospital", "hospital", "red", "Mostrar la ruta al hospital más cercano")
park_button = create_button_with_style("Parque", "tree", "green", "Mostrar la ruta al parque más cercano")

# Contenedor para los detalles del piso
details_output = Output()

# Añadir el botón cerrar y los botones de rutas al panel
output_panel.children = [close_button, details_output, school_button, hospital_button, park_button]

close_button.on_click(close_panel_callback)

# Crear una lista de marcadores para los pisos
markers = []

# Lista para almacenar las rutas actuales
current_routes = []

# Conectar el selector de barrios con la función de actualización
barrio_selector.observe(update_map, names="value")

# Añadir el selector al mapa como un WidgetControl en la parte superior izquierda
barrio_selector_control = WidgetControl(widget=barrio_selector, position="topleft")  # Cambia "topright" por "topleft" si es necesario
m.add_control(barrio_selector_control)

# Contenedor principal (mapa y panel superpuestos)
ui = HBox([m, output_panel])
ui.layout = Layout(width="100%", height="100vh")

# Mostrar la interfaz
display(ui)